# YOLO Object Detecion with Tensorflow Object Detection API

In [3]:
# Prerequisites and Dependencies:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

from ultralytics import YOLO

# Dependencies
# !pip install ultralytics
# !pip install tf_slim 
# !pip install tensorflow-object-detection-api

### Load the Model

In [4]:
# Load the Model
model = YOLO('yolov8n.pt')


### Load an Image

In [6]:
IMAGE_PATH = '../images/motorcycle_640x640.jpg'

image = Image.open(IMAGE_PATH).convert('RGB')
W, H = image.size
image_np = np.array(image)

### Run Inference

In [12]:
results = model.predict(source=image_np, conf=0.25, imgsz=640, verbose=False)
r = results[0]

xyxy = r.boxes.xyxy.cpu().numpy() if r.boxes is not None else np.empty((0, 4))
scores = r.boxes.conf.cpu().numpy() if r.boxes is not None else np.empty((0,))
classes = r.boxes.cls.cpu().numpy().astype(np.int64) if r.boxes is not None else np.empty((0,), dtype=np.int64)

print(f"xyxy: {xyxy}")
print(f"scores: {scores}")
print(f"classes: {classes}")


xyxy: [[     62.852      306.64      521.94      626.65]
 [     133.91      276.33      310.83      576.28]]
scores: [    0.73594     0.38659]
classes: [3 0]


### Visualize

In [ ]:
# Convert to TensorFlow OD API style for visualization_utils
# TF-OD expects boxes as [ymin, xmin, ymax, xmax] in *normalized* coordinates
if xyxy.shape[0] > 0:
    xmin = xyxy[:, 0] / W
    ymin = xyxy[:, 1] / H
    xmax = xyxy[:, 2] / W
    ymax = xyxy[:, 3] / H
    boxes_tf = np.stack([ymin, xmin, ymax, xmax], axis=1).astype(np.float32)
else:
    boxes_tf = np.empty((0, 4), dtype=np.float32)

scores_tf = scores.astype(np.float32)
# TF-OD label ids typically start at 1; YOLO classes start at 0.
classes_tf = (classes + 1).astype(np.int32)




In [13]:
names = model.model.names  # dict like {0: 'person', 1: 'bicycle', ...}
# Create a label map dict that label_map_util understands
label_map = {i + 1: {'id': i + 1, 'name': names[i]} for i in range(len(names))}
category_index = label_map  # label_map_util.create_category_index would accept a similar structure

print(label_map)

{1: {'id': 1, 'name': 'person'}, 2: {'id': 2, 'name': 'bicycle'}, 3: {'id': 3, 'name': 'car'}, 4: {'id': 4, 'name': 'motorcycle'}, 5: {'id': 5, 'name': 'airplane'}, 6: {'id': 6, 'name': 'bus'}, 7: {'id': 7, 'name': 'train'}, 8: {'id': 8, 'name': 'truck'}, 9: {'id': 9, 'name': 'boat'}, 10: {'id': 10, 'name': 'traffic light'}, 11: {'id': 11, 'name': 'fire hydrant'}, 12: {'id': 12, 'name': 'stop sign'}, 13: {'id': 13, 'name': 'parking meter'}, 14: {'id': 14, 'name': 'bench'}, 15: {'id': 15, 'name': 'bird'}, 16: {'id': 16, 'name': 'cat'}, 17: {'id': 17, 'name': 'dog'}, 18: {'id': 18, 'name': 'horse'}, 19: {'id': 19, 'name': 'sheep'}, 20: {'id': 20, 'name': 'cow'}, 21: {'id': 21, 'name': 'elephant'}, 22: {'id': 22, 'name': 'bear'}, 23: {'id': 23, 'name': 'zebra'}, 24: {'id': 24, 'name': 'giraffe'}, 25: {'id': 25, 'name': 'backpack'}, 26: {'id': 26, 'name': 'umbrella'}, 27: {'id': 27, 'name': 'handbag'}, 28: {'id': 28, 'name': 'tie'}, 29: {'id': 29, 'name': 'suitcase'}, 30: {'id': 30, 'name'

In [11]:
from object_detection.utils import visualization_utils as viz_utils

image_vis = image_np.copy()
viz_utils.visualize_boxes_and_labels_on_image_array(
    image_vis,
    boxes_tf,
    classes_tf,
    scores_tf,
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=100,
    min_score_thresh=0.25,
    agnostic_mode=False,
    line_thickness=3
)

plt.figure(figsize=(12, 8))
plt.imshow(image_vis)
plt.axis('off')
plt.show()

AttributeError: 'FreeTypeFont' object has no attribute 'getsize'